In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [52]:
!pip install np_utils

In [21]:
import numpy as np
import pandas as pd
from plotly.offline import iplot
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.metrics import Recall, 
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from tqdm import tqdm

In [3]:
politifact = pd.read_json(r'./data/politifact.json')
snopes = pd.read_json(r'./data/snopes.json')

In [4]:
# column names
poli_cols = politifact.columns.to_list()
snop_cols = snopes.columns.to_list()

poli_row_count, poli_col_count = politifact.shape
snop_row_count, snop_col_count = snopes.shape

In [5]:
common_cols = set(poli_cols) & set(snop_cols)

td = pd.concat([snopes[list(common_cols)], politifact[list(common_cols)]])
td = td.reindex(columns = ['doc', 'claim', 'factchecker', 'url', 'sources', 'topic', 'published' ,'label' ])


In [6]:
td2 =td.dropna(subset=['doc','claim','label'])

In [7]:
print(td2.shape, td.shape)

(22298, 8) (22298, 8)


In [8]:
td2['label']=td2.label.map(lambda x: x.lower())

In [9]:
valid_labels = ['false','true','half-true','mostly-true','barely-true','pants-fire','mixture']

td2 = td2[td2.label.isin(valid_labels)]

In [10]:
td2 = td2.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):

    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
td2['claim'] = td2['claim'].apply(clean_text)
td2['doc'] = td2['doc'].apply(clean_text)

In [11]:
td2['body'] = td2['doc'].str.cat(td2['claim'], sep=' ')

In [12]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(td2['body'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 223445 unique tokens.


In [13]:
X = tokenizer.texts_to_sequences(td2['body'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (20874, 250)


In [14]:
Y = pd.get_dummies(td2['label']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (20874, 7)


In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(18786, 250) (18786, 7)
(2088, 250) (2088, 7)


In [24]:
EMBEDDING_DIM

100

In [16]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 250, 100)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 7)                 707       
                                                                 
Total params: 5,081,107
Trainable params: 5,081,107
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
epochs = 10
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
     

Epoch 1/10
265/265 [==============================] - 55s 198ms/step - loss: 0.9797 - accuracy: 0.6451 - val_loss: 0.2755 - val_accuracy: 0.9218
Epoch 2/10
265/265 [==============================] - 52s 196ms/step - loss: 0.2125 - accuracy: 0.9317 - val_loss: 0.2230 - val_accuracy: 0.9191
Epoch 3/10
265/265 [==============================] - 52s 197ms/step - loss: 0.1055 - accuracy: 0.9694 - val_loss: 0.2558 - val_accuracy: 0.9058
Epoch 4/10
265/265 [==============================] - 52s 198ms/step - loss: 0.0639 - accuracy: 0.9839 - val_loss: 0.2542 - val_accuracy: 0.9148
Epoch 5/10
265/265 [==============================] - 52s 197ms/step - loss: 0.0287 - accuracy: 0.9928 - val_loss: 0.3073 - val_accuracy: 0.9244


In [29]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(accuracy,loss)

66/66 [==============================] - 2s 27ms/step - loss: 0.2959 - accuracy: 0.9296
0.9295976758003235 0.2959437370300293
